In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bankok-traffy-csv/bangkok_traffy.csv
/kaggle/input/fondue-data-scape/fondue_scraping.csv
/kaggle/input/fondue-data-scape/fondue_data_scape.csv


In [2]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/kaggle/input/bankok-traffy-csv/bangkok_traffy.csv')
df_scrape= pd.read_csv('/kaggle/input/fondue-data-scape/fondue_scraping.csv')

In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

# Sample target departments and Bangkok variants (fill in with your actual data)
target_departments = {
    'สำนักงานตำรวจแห่งชาติ',
    'กองบัญชาการตำรวจนครบาล',
    'สำนักการโยธา',
    'สำนักงานก่อสร้างและบูรณะ',
    'การไฟฟ้านครหลวง',
    'สำนักการจราจรและขนส่ง',
    'สำนักสิ่งแวดล้อม',
    'สำนักการระบายน้ำ',
    'กองบังคับการตำรวจนครบาล',
    'กลุ่มงานสัญญาณไฟจราจร',
    'กสทช.',
    'กลุ่มงานบริหารทรัพย์สิน',
    'ขสมก.',
    'การประปานครหลวง',
    'เขต',
    'สน.'
}
bangkok_variants = {
    'กรุงเทพมหานคร',
    'จังหวัดกรุงเทพมหานคร',
}
bangkok_districts = {
    'สาทร', 'ประเวศ', 'ดุสิต', 'บางเขน', 'ลาดพร้าว', 'ราชเทวี',
    'บางซื่อ', 'บึงกุ่ม', 'คลองเตย', 'ทุ่งครุ', 'ดอนเมือง',
    'วังทองหลาง', 'ดินแดง', 'พญาไท', 'ยานนาวา', 'ตลิ่งชัน', 'หลักสี่',
    'บางขุนเทียน', 'ห้วยขวาง', 'พระนคร', 'จอมทอง', 'บางกะปิ',
    'บางคอแหลม', 'หนองจอก', 'บางแค', 'จตุจักร', 'บางพลัด', 'บางบอน',
    'ภาษีเจริญ', 'ราษฎร์บูรณะ', 'คันนายาว', 'บางกอกน้อย', 'วัฒนา',
    'คลองสาน', 'ลาดกระบัง', 'พระโขนง', 'หนองแขม', 'สวนหลวง',
    'คลองสามวา', 'สัมพันธวงศ์', 'ปทุมวัน', 'สายไหม', 'บางนา', 'ธนบุรี',
    'ป้อมปราบศัตรูพ่าย', 'สะพานสูง', 'มีนบุรี', 'ทวีวัฒนา', 'บางรัก',
    'บางกอกใหญ่'
}

# Custom transformers
class DateTimeConverter(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.cols:
            X[col] = pd.to_datetime(X[col], errors='coerce')
        return X


class ResolutionTimeCalculator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X['resolution_time'] = (X['last_activity'] - X['timestamp']).dt.total_seconds() / 3600
        return X


class OutlierRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        Q1 = X['resolution_time'].quantile(0.25)
        Q3 = X['resolution_time'].quantile(0.75)
        IQR = Q3 - Q1
        self.lower_bound = Q1 - 1.5 * IQR
        self.upper_bound = Q3 + 1.5 * IQR
        return self

    def transform(self, X):
        return X[(X['resolution_time'] >= self.lower_bound) & (X['resolution_time'] <= self.upper_bound)].copy()


class DepartmentMatcher(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def match_department(org_name):
            for keyword in target_departments:
                if pd.notnull(org_name) and keyword in org_name:
                    return keyword
            return None

        X = X.copy()
        X['organization'] = X['organization'].apply(match_department)
        return X


class TextCleaner(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        def clean_text(text):
            if pd.isna(text):
                return ""
            text = re.sub(r'[^a-zA-Z0-9\u0E00-\u0E7F\s]', '', text)
            text = re.sub(r'\s+', ' ', text).strip()
            return text

        X = X.copy()
        X['comment'] = X['comment'].apply(clean_text)
        return X


# The full pipeline
pipeline = Pipeline(steps=[
    ('convert_dates', DateTimeConverter(['timestamp', 'last_activity'])),
    ('drop_na_essential', FunctionTransformer(
        lambda df: df.dropna(subset=['ticket_id', 'comment', 'type', 'timestamp', 'last_activity', 'organization', 'district', 'state']),
        validate=False,
        feature_names_out='one-to-one'
    )),
    ('drop_duplicates', FunctionTransformer(
        lambda df: df.drop_duplicates(),
        validate=False,
        feature_names_out='one-to-one'
    )),
    ('calc_resolution_time', ResolutionTimeCalculator()),
    ('remove_outliers', OutlierRemover()),
    ('impute_star', FunctionTransformer(
        lambda df: df.assign(star=df['star'].fillna(df['star'].mean())),
        validate=False,
        feature_names_out='one-to-one'
    )),
    ('match_department', DepartmentMatcher()),
    ('drop_org_na', FunctionTransformer(
        lambda df: df.dropna(subset=['organization']),
        validate=False,
        feature_names_out='one-to-one'
    )),
    ('clean_comment', TextCleaner()),
    ('filter_province', FunctionTransformer(
        lambda df: df[df['province'].isin(bangkok_variants)].copy(),
        validate=False,
        feature_names_out='one-to-one'
    )),
    ('filter_district', FunctionTransformer(
        lambda df: df[df['district'].isin(bangkok_districts)].copy(),
        validate=False,
        feature_names_out='one-to-one'
    ))
])

In [4]:
df_cleaned = pipeline.fit_transform(df)
df_scrape_cleaned = pipeline.fit_transform(df_scrape)

In [5]:
print("df_cleaned:", df_cleaned.shape)
print("df_scapre_cleaned:", df_scrape_cleaned.shape)

df_cleaned: (660994, 17)
df_scapre_cleaned: (14321, 15)


In [6]:
df_combined = pd.concat([df_cleaned, df_scrape_cleaned], ignore_index=True)

In [7]:
df_combined['district'].value_counts()

district
จตุจักร              32465
ประเวศ               23833
คลองเตย              22809
วัฒนา                22405
บางกะปิ              21429
บางแค                21384
ปทุมวัน              19819
บางเขน               19791
บางขุนเทียน          18237
ราชเทวี              16859
ดินแดง               16221
พระนคร               16127
บางกอกน้อย           16021
บึงกุ่ม              15632
สาทร                 14843
บางซื่อ              14707
ลาดพร้าว             14359
ธนบุรี               14317
ห้วยขวาง             13956
สวนหลวง              13837
ลาดกระบัง            13655
วังทองหลาง           13352
คลองสามวา            13269
พญาไท                12864
บางนา                12424
ภาษีเจริญ            12231
หนองจอก              12192
บางพลัด              12188
หลักสี่              11957
สายไหม               11933
มีนบุรี              11738
ตลิ่งชัน             11711
ยานนาวา              10924
ป้อมปราบศัตรูพ่าย    10773
บางรัก               10704
พระโขนง              10636
บางคอแหลม          

In [8]:
df_combined['organization'].value_counts()

organization
เขต                         636978
สำนักงานก่อสร้างและบูรณะ     37616
สำนักสิ่งแวดล้อม               275
กองบัญชาการตำรวจนครบาล         141
กลุ่มงานสัญญาณไฟจราจร          103
สำนักการจราจรและขนส่ง           91
สำนักการโยธา                    49
สำนักการระบายน้ำ                47
การประปานครหลวง                 11
ขสมก.                            2
การไฟฟ้านครหลวง                  1
กสทช.                            1
Name: count, dtype: int64

In [9]:
df_combined.to_csv('cleaned_data.csv', index=False)

In [10]:
df_combined.tail()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,resolution_time
675310,2025-9MDMC4,{ผิดกฎจราจร},เขต,วันที่ 18 เมย 2568 ยังคงมี รถตุ๊กตุ๊ก จอด ซึ่ง...,NaN,NaN,"100.52773, 13.76707",ป้ายรถประจำทาง โรงพยาบาลรามา 1 แขวงทุ่งพญาไท เ...,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,2025-04-18 09:17:05,ส่งต่อ(ใหม่),4.270893,0,2025-04-18 09:34:12,0.285278
675311,2025-V6JH9K,{ต้นไม้},เขต,ต้นไม้ยื่นกิ่งก้านยาวเข้าบ้าน,NaN,NaN,"100.48312, 13.71854",132 ถ. ราชพฤกษ์ แขวงบุคคโล เขตธนบุรี กรุงเทพมห...,บุคคโล,ธนบุรี,กรุงเทพมหานคร,2025-04-18 09:16:27,เสร็จสิ้น,5.000000,0,2025-04-18 22:29:51,13.223333
675312,2025-U2NYVN,{ผิดกฎจราจร},เขต,จุดนี้มีรถกระบะสีกรม ทะเบียน ณต3127 จอดขวางเส้...,NaN,NaN,"100.59882, 13.88766",54 ซอย พหลโยธิน 63 แยก 2 แขวงอนุสาวรีย์ เขตบาง...,อนุสาวรีย์,บางเขน,จังหวัดกรุงเทพมหานคร,2025-04-18 09:15:04,เสร็จสิ้น,5.000000,0,2025-04-19 16:16:25,31.022500
675313,88V7MX,{สัตว์},เขต,ศูนย์เรื่องราวร้องทุกข์ ได้รับการประสานผ่านระบ...,NaN,NaN,"100.6605, 13.89515",สำนักงานเขตสายไหม ถนนสุขาภิบาล ๕ แขวงออเงิน เข...,ออเงิน,สายไหม,จังหวัดกรุงเทพมหานคร,2025-04-18 09:13:41,เสร็จสิ้น,4.270893,0,2025-04-23 22:15:21,133.027778
675314,2025-K9PNFH,{เสียง},เขต,ข้าพเจ้าขอเรียนแจ้งว่า มีการต่อเติมและก่อสร้าง...,NaN,NaN,"100.52153, 13.74182",150/13 ซอย จรัญเมือง แขวงรองเมือง เขตปทุมวัน ก...,รองเมือง,ปทุมวัน,จังหวัดกรุงเทพมหานคร,2025-04-18 09:08:56,เสร็จสิ้น,4.270893,0,2025-04-22 12:35:49,99.448056
